In [ ]:
!pip install sentence-transformers
!pip install faiss-cpu


In [19]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle
import os
import pandas as pd

In [20]:



class MessageManager:
    def __init__(self, model_name='paraphrase-multilingual-MiniLM-L12-v2', index_path='faiss.index', data_path='data.pkl'):
        self.model = SentenceTransformer(model_name)
        self.messages = []
        self.embeddings = np.empty((0, self.model.get_sentence_embedding_dimension()), dtype='float32')
        self.dimension = self.model.get_sentence_embedding_dimension()
        self.index_path = index_path
        self.data_path = data_path
        self.index = faiss.IndexFlatL2(self.dimension)
        self.load_data()
    
    def add_new_message(self, new_message):

        new_embedding = self.model.encode([new_message]).astype('float32')
        self.index.add(new_embedding)
        self.messages.append(new_message)
        self.embeddings = np.vstack([self.embeddings, new_embedding])
       
    
    def is_similar(self, new_message, threshold=0.8):

        if len(self.messages) == 0:
            return False , False
        new_embedding = self.model.encode([new_message]).astype('float32')
        
        D, I = self.index.search(new_embedding, k=1)
        nearest_index = I[0][0]
        nearest_message = self.messages[nearest_index]

        existing_embedding = self.embeddings[I[0][0]]
        similarity = np.dot(new_embedding, existing_embedding) / (np.linalg.norm(new_embedding) * np.linalg.norm(existing_embedding))

        similar = similarity > threshold

        return nearest_message , similar[0]
    
    def save_data(self):
        faiss.write_index(self.index, self.index_path)
        with open(self.data_path, 'wb') as f:
            pickle.dump({'messages': self.messages, 'embeddings': self.embeddings}, f)
       
    
    def load_data(self):
        if os.path.exists(self.data_path) and os.path.exists(self.index_path):
            with open(self.data_path, 'rb') as f:
                data = pickle.load(f)
                self.messages = data['messages']
                self.embeddings = data['embeddings']
            self.index = faiss.read_index(self.index_path)

    
    def shutdown(self):
        self.save_data()


manager = MessageManager()
manager.shutdown()



Файли даних не знайдено. Починаємо з порожньої бази.


In [21]:
def check_message_in_date_range(df, start_date, end_date, similar_message):
    df["date"] = pd.to_datetime(df["date"])
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    filtered_df = df[(df["date"] >= start_date) & (df["date"] <= end_date)]
    #print(filtered_df)
    return any(filtered_df["message"].str.contains(similar_message, case=False, na=False))

In [22]:
df_current = pd.read_csv("/Users/ulanagusar/Desktop/ML_week/table1.csv",delimiter=";")

In [23]:
df_current

,TelegramPostInfoID,Message,Channel,MessageDate
0,10,США цього тижня перевезли близько 90 ракет до ...,STERNENKO,2025-01-29 13:02:52.000
1,11,У аеропорту Мурманська запровадили план «ковьо...,STERNENKO,2025-01-29 10:18:57.000
2,12,Миколаїв під атакою балістики.,STERNENKO,2025-01-28 20:02:51.000
3,13,росія втратила понад 20 000 одиниць військової...,STERNENKO,2025-01-28 19:28:10.000


In [25]:
file_path_stable = "/Users/ulanagusar/Desktop/ML_week/messages_data.csv"  
# df_current = pd.read_csv("/Users/ulanagusar/Desktop/ML_week/table1.csv")

def rm_dublicates(df_current , file_path_stable):
        
        manager = MessageManager()
        manager.shutdown()
        messages = df_current["Message"].to_list()
        ids = df_current["TelegramPostInfoID"].to_list()
        dates = df_current["MessageDate"].to_list()
        df_stable = pd.read_csv(file_path_stable )

        start_date = "2024-02-20"
        end_date = "2026-03-08"
        res_id = []
        threshold = 0.8
        for i in range(len(messages)):
            message = messages[i]
            id = ids[i]
            date = dates[i]
            nearest_message , similar = manager.is_similar(message, threshold)
            if similar == False :
                manager.add_new_message(message)
                new_row = [(date, message)]
                df_new = pd.DataFrame(new_row, columns=["date", "message"]) 
                df_new.to_csv(file_path_stable, mode='a', index=False, header=False, encoding="utf-8")
                df_stable = pd.read_csv(file_path_stable)
                res_id.append(id)
            else:
                message_in_date_range = check_message_in_date_range(df_stable, start_date, end_date, nearest_message)
                if message_in_date_range == False :
                    manager.add_new_message(message)
                    new_row = [(date, message)]
                    df_new = pd.DataFrame(new_row, columns=["date", "message"]) 
                    df_new.to_csv(file_path_stable, mode='a', index=False, header=False, encoding="utf-8")
                    df_stable = pd.read_csv(file_path_stable)
                    res_id.append(id)
        manager.shutdown()
        return res_id


In [29]:
res_id = rm_dublicates(df_current , file_path_stable)

Дані завантажено успішно!


In [28]:
res_id

[10, 11, 12, 13]